In [ ]:
%pip -q uninstall -y transformers tokenizers datasets accelerate huggingface_hub protobuf numpy pyarrow
%pip -q install \
  "protobuf<5.0.0" \
  "transformers==4.44.2" \
  "datasets==2.20.0" \
  "accelerate==0.34.2" \
  "tokenizers>=0.19.1" \
  "huggingface_hub>=0.24" \
  "safetensors>=0.4" \
  "sentencepiece>=0.1.99" \
  "numpy==1.26.4" \
  "pyarrow==15.0.2"

Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 71.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 87.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 46.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 69.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━

In [ ]:
import os
os.kill(os.getpid(), 9)

In [3]:
import transformers, datasets, accelerate, google.protobuf as gp, numpy, pyarrow
import torch, sys, platform
print("transformers:", transformers.__version__)
print("datasets:", datasets.__version__)
print("accelerate:", accelerate.__version__)
print("protobuf:", gp.__version__)
print("numpy:", numpy.__version__)
print("pyarrow:", pyarrow.__version__)
print("python:", sys.version.split()[0], "| torch:", torch.__version__, "| cuda:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("gpu:", torch.cuda.get_device_name(0))

transformers: 4.44.2
datasets: 2.20.0
accelerate: 0.34.2
protobuf: 4.25.8
numpy: 1.26.4
pyarrow: 15.0.2
python: 3.11.13 | torch: 2.6.0+cu124 | cuda: False


In [ ]:
import os, json, numpy as np, torch
from datasets import load_from_disk
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          TrainingArguments, Trainer, EarlyStoppingCallback)

BASE_DIR = "/kaggle/working"
LANG_TAG = "en"
SAVE_LANG_DIR = f"{BASE_DIR}/{LANG_TAG}"
TOK_DIR = f"{SAVE_LANG_DIR}/tok_xlmr_256"

SEED = 42
torch.manual_seed(SEED); 
if torch.cuda.is_available(): torch.cuda.manual_seed_all(SEED)

2025-12-04 18:59:00.386940: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764874740.663906     114 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764874740.743095     114 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
tok = load_from_disk(TOK_DIR)
with open(f"{SAVE_LANG_DIR}/label2id.json","r",encoding="utf-8") as f: label2id = json.load(f)
with open(f"{SAVE_LANG_DIR}/id2label.json","r",encoding="utf-8") as f: id2label = json.load(f)
num_labels = len(label2id)

MODEL_NAME = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
TOK_DIR = f"es/tok_xlmr_256"
tok = load_from_disk(TOK_DIR)

train_full = tok["train"]
test_full  = tok["test"]

print("Spanish Original train size:", len(train_full))
print("Spanish Original test size :", len(test_full))

labels = np.array(tok["train"]["labels"])
print(np.bincount(labels))


Spanish Original train size: 6087
Spanish Original test size : 7555
[2029 2029 2029]


In [11]:
TOK_DIR = f"pt/tok_xlmr_256"
tok = load_from_disk(TOK_DIR)

train_full = tok["train"]
test_full  = tok["test"]

print("Portuguese Original train size:", len(train_full))
print("Portuguese Original test size :", len(test_full))

labels = np.array(tok["train"]["labels"])
print(np.bincount(labels))


Portuguese Original train size: 2832
Portuguese Original test size : 3945
[944 944 944]


In [12]:
TOK_DIR = f"hi/tok_xlmr_256"
tok = load_from_disk(TOK_DIR)

train_full = tok["train"]
test_full  = tok["test"]

print("Hindi Original train size:", len(train_full))
print("Hindi Original test size :", len(test_full))

labels = np.array(tok["train"]["labels"])
print(np.bincount(labels))


Hindi Original train size: 8100
Hindi Original test size : 3958
[2700 2700 2700]


In [13]:
TOK_DIR = f"id/tok_xlmr_256"
tok = load_from_disk(TOK_DIR)

train_full = tok["train"]
test_full  = tok["test"]

print("Indonesian Original train size:", len(train_full))
print("Indonesian Original test size :", len(test_full))

labels = np.array(tok["train"]["labels"])
print(np.bincount(labels))


Indonesian Original train size: 3792
Indonesian Original test size : 2873
[1264 1264 1264]
